In [5]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("List available databases and tables").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.catalog.listDatabases()

[Database(name='2022_10_22', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='2023_04_01', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse')]

In [6]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark.sql("USE 2023_04_01")
    # Assuming that 'ticker' is a valid Python variable

# query = f"""SELECT ticker_region FROM sym_ticker_region WHERE ticker_region LIKE "%-US" """
# df = spark.sql(query)
# df = df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
# ticker_list = df.collect()
# print(len(ticker_list))


DataFrame[]

In [66]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import when


#fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))

def query(ticker,date):
    query = f"""SELECT d.ticker_region, a.date, FF_PRICE_CLOSE_FP
                FROM FF_BASIC_AF a 
                LEFT JOIN sym_ticker_region d ON d.fsym_id = a.fsym_id 
                WHERE d.ticker_region = "{ticker}-US" AND a.date = "{date}" 
                """

    fund_df = spark.sql(query)
    fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
    
    return fund_df

def pull_prev_data(df):
    df['prev4'] = 0
    df['prev1']=0


    for index, row in df.iterrows():
        ticker_value = row['Ticker']
        date_value = row['Implosion_Date - 4yrs']
        dv2 = row['Implosion_Date - 1yrs']

        result_df = query(ticker_value, date_value).toPandas()
        r2 = query(ticker_value, dv2).toPandas()

        if not result_df.empty:
            fourth_price = result_df['FF_PRICE_CLOSE_FP'].iloc[0]

            # Update 'prev' column based on conditions
            condition = (
                (df['Ticker'].values == result_df['ticker_region'].values) &
                (df['Implosion_Date - 4yrs'].values == result_df['date'].values)
            )
            df.loc[condition, 'prev4'] = fourth_price
            
        if not r2.empty:
            first_price = r2['FF_PRICE_CLOSE_FP'].iloc[0]

            # Update 'prev' column based on conditions
            condition = (
                (df['Ticker'].values == r2['ticker_region'].values) &
                (df['Implosion_Date - 1yrs'].values == r2['date'].values)
            )
            df.loc[condition, 'prev1'] = first_price
            print(df.head())
    df = spark.createDataFrame(df)
    df.show()
    df=df.orderBy('Ticker','Implosion_Date')

    return df



def create_df(filename):
    df = pd.read_csv(filename, index_col=None, usecols=['Ticker', 'Implosion_Date'])
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date']).dt.date
    df['Implosion_Date - 1yrs'] = df['Implosion_Date'] - pd.DateOffset(years=1)
    df['Implosion_Date - 1yrs'] =  pd.to_datetime(df['Implosion_Date - 1yrs']).dt.date
    df['Implosion_Date - 1yrs'] =  df['Implosion_Date - 1yrs'].apply(lambda x: x.replace(month=12, day=31))
    df['Implosion_Date - 4yrs'] = df['Implosion_Date'] - pd.DateOffset(years=4)
    df['Implosion_Date - 4yrs'] =  pd.to_datetime(df['Implosion_Date - 4yrs']).dt.date
    df['Implosion_Date - 4yrs'] =  df['Implosion_Date - 4yrs'].apply(lambda x: x.replace(month=12, day=31))
    df=pull_prev_data(df)
    return df
    
    

df = create_df('imploded_tickers_dates.csv')
df.show(100)

  Ticker Implosion_Date Implosion_Date - 1yrs Implosion_Date - 4yrs     prev4  \
0  EMCMF     2022-09-04            2021-12-31            2018-12-31  0.507639   
1  MGAWF     2022-10-30            2021-12-31            2018-12-31  0.000000   
2   SWNM     2022-04-10            2021-12-31            2018-12-31  0.000000   
3  MBGCF     2016-10-02            2015-12-31            2012-12-31  0.000000   
4  MBGCF     2018-05-06            2017-12-31            2014-12-31  0.000000   

      prev1  
0  0.490836  
1  0.000000  
2  0.000000  
3  0.000000  
4  0.000000  
  Ticker Implosion_Date Implosion_Date - 1yrs Implosion_Date - 4yrs     prev4  \
0  EMCMF     2022-09-04            2021-12-31            2018-12-31  0.507639   
1  MGAWF     2022-10-30            2021-12-31            2018-12-31  0.090330   
2   SWNM     2022-04-10            2021-12-31            2018-12-31  0.000000   
3  MBGCF     2016-10-02            2015-12-31            2012-12-31  0.000000   
4  MBGCF     2018-05-06 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#spark.stop()